<a href="https://colab.research.google.com/github/yasirfaizahmed/ML_Notebooks/blob/master/NumberPlate_detection_using_YOLOv8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# license plate object detection using YOLOv8

#### dataset : https://huggingface.co/datasets/keremberke/license-plate-object-detection


In [1]:
! pip install datasets ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 913.6/913.6 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from datasets import load_dataset

ds = load_dataset("keremberke/license-plate-object-detection", "full")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.88k [00:00<?, ?B/s]

license-plate-object-detection.py:   0%|          | 0.00/6.11k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/168M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/46.3M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/22.6M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6176 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1765 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/882 [00:00<?, ? examples/s]

In [3]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import cv2
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
df_train = ds['train'].to_pandas()
df_val = ds['validation'].to_pandas()
df_test = ds['test'].to_pandas()
df_train.info()
df_train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6176 entries, 0 to 6175
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   image_id  6176 non-null   int64 
 1   image     6176 non-null   object
 2   width     6176 non-null   int32 
 3   height    6176 non-null   int32 
 4   objects   6176 non-null   object
dtypes: int32(2), int64(1), object(2)
memory usage: 193.1+ KB


,image_id,image,width,height,objects
0,6152,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,472,303,"{'id': [6388], 'area': [2604], 'bbox': [[261.0..."
1,5500,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,472,303,"{'id': [5719], 'area': [742], 'bbox': [[263.0,..."
2,1188,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,400,240,"{'id': [1231], 'area': [2701], 'bbox': [[147.0..."
3,3524,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,540,405,"{'id': [3651], 'area': [800], 'bbox': [[217.0,..."
4,5049,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,472,303,"{'id': [5242], 'area': [2112], 'bbox': [[145.0..."


In [5]:
# Load a pre-trained YOLOv8 model
model = YOLO('yolov8n.pt')  # 'yolov8n.pt' is a small model, you can choose others like 'yolov8

100%|██████████| 6.25M/6.25M [00:00<00:00, 76.2MB/s]


In [6]:
import os
import cv2
print(df_train["objects"][0])


{'id': array([6388]), 'area': array([2604]), 'bbox': array([array([        261,         203,          93,          28], dtype=float32)], dtype=object), 'category': array([0])}


In [7]:
# Output directories
image_dir = 'dataset/images/train'
label_dir = 'dataset/labels/train'
os.makedirs(image_dir, exist_ok=True)
os.makedirs(label_dir, exist_ok=True)

for idx, row in df_train.iterrows():
    # Convert byte string to image
    image = np.frombuffer(row['image'].get('bytes'), dtype=np.uint8)
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)

    if image is not None:
      image_height, image_width = image.shape[:2]
    else:
      continue

    # Save image
    image_path = os.path.join(image_dir, f'image_{idx}.jpg')
    cv2.imwrite(image_path, image)

    # Convert bbox to YOLO format
    # Extract bounding box and class ID
    bbox = row['objects'].get("bbox")  # [x_min, y_min, width, height]
    class_id = 0 # Class ID

    # Normalize bbox coordinates
    x_min, y_min, width, height = bbox[0]
    x_center = (x_min + width / 2) / image_width
    y_center = (y_min + height / 2) / image_height
    bbox_width = width / image_width
    bbox_height = height / image_height

    # Save label file
    label_path = os.path.join(label_dir, f'image_{idx}.txt')
    with open(label_path, 'w') as f:
        f.write(f'{class_id} {x_center} {y_center} {bbox_width} {bbox_height}\n')

In [8]:
# Output directories
image_dir = 'dataset/images/val'
label_dir = 'dataset/labels/val'
os.makedirs(image_dir, exist_ok=True)
os.makedirs(label_dir, exist_ok=True)

for idx, row in df_val.iterrows():
    # Convert byte string to image
    image = np.frombuffer(row['image'].get('bytes'), dtype=np.uint8)
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)

    if image is not None:
      image_height, image_width = image.shape[:2]
    else:
      continue

    # Save image
    image_path = os.path.join(image_dir, f'image_{idx}.jpg')
    cv2.imwrite(image_path, image)

    # Convert bbox to YOLO format
    # Extract bounding box and class ID
    bbox = row['objects'].get("bbox")  # [x_min, y_min, width, height]
    class_id = 0  # Class ID


    # Normalize bbox coordinates
    x_min, y_min, width, height = bbox[0]
    x_center = (x_min + width / 2) / image_width
    y_center = (y_min + height / 2) / image_height
    bbox_width = width / image_width
    bbox_height = height / image_height

    # Save label file
    label_path = os.path.join(label_dir, f'image_{idx}.txt')
    with open(label_path, 'w') as f:
        f.write(f'{class_id} {x_center} {y_center} {bbox_width} {bbox_height}\n')

In [11]:
# Train the model using the YAML file
results = model.train(data="dataset.yaml", epochs=75, imgsz=640, batch=16)

Ultralytics 8.3.68 🚀 Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=dataset.yaml, epochs=75, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes

train: Scanning /content/dataset/labels/train.cache... 6176 images, 0 backgrounds, 0 corrupt: 100%|██████████| 6176/6176 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/dataset/labels/val.cache... 1765 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1765/1765 [00:00<?, ?it/s]


Plotting labels to runs/detect/train3/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train3
Starting training for 75 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      75/75      4.01G     0.9056     0.3249     0.9883         16        640: 100%|██████████| 386/386 [00:34<00:00, 11.22it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 56/56 [00:06<00:00,  9.08it/s]

                   all       1765       1765       0.99      0.972      0.991      0.744



75 epochs completed in 0.876 hours.
Optimizer stripped from runs/detect/train3/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train3/weights/best.pt, 6.2MB

Validating runs/detect/train3/weights/best.pt...
Ultralytics 8.3.68 🚀 Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 56/56 [00:06<00:00,  8.10it/s]


                   all       1765       1765       0.99      0.974      0.993      0.744
Speed: 0.1ms preprocess, 0.4ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/detect/train3
